In [2]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
# import json
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd

from matplotlib import colors

import pickle

import ast
import scipy

import seaborn as sns

from SGanalysis.SGobject import SGobject 

import matplotlib
%matplotlib inline
# matplotlib.use('Qt5Agg')
# %matplotlib qt

from itertools import combinations

from sklearn.cluster import AgglomerativeClustering as AggCluster

import sys
import os

%load_ext autoreload
%autoreload 2

tools_path  = '/Users/grantkinsler/Documents/Penn/Research/SpatialBarcodes/analysis/tools/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

In [3]:
# all_files = []

# # filepath  = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/Shared_Grant/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'
# filepath = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/'

# # roi_2
# roi_name = 'roi_2'

# for roi in [1,2,3]:
#     all_files.append(f'{filepath}roi_{roi_name})
                     
# all_files.append(')
    
### roi_name: [segmentation_file, spots_file, out_path]
roi_file_paths = {
                'roi_1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/segmentations/nuclei_20240604_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports'
                           },
                'roi_2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/segmentations/nuclei_20240529_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports'
                           },
                'roi_3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/segmentations/nuclei_20240604_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports'
                           },
                'timezero_roi_1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/segmentations/segmentation_20240513_nuclei.tiff',
                              # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                              'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/transcripts/20240517_segmentation_withRefid.csv',
                              'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/exports'
                              },
                  'timezero_roi_2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/segmentations/segment_091924_nuclei.tiff',
                            # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                            'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/transcripts/transcripts.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/exports'
                            },
                    'timezero_roi_3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/segmentations/segment_20240921_nuclei.tiff',
                              # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                              'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/transcripts/transcripts.csv',
                              'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/exports'
                              },
                'run2_roi_2':{'segmentation_file':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/segmentation.tiff',
                            'spots_file':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/transcripts.csv',
                            'out_path':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/',
                            },
                'run2_roi_3':{'segmentation_file':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/segmentation.tiff',
                            'spots_file':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/transcripts.csv',
                            'out_path':'/Volumes/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/',
                            },
                  
                  }


# /roi_2

In [27]:
roi_barcodes = {}

for roi_name,files in roi_file_paths.items():

    out_path = files['out_path']

    unclustered_df = pd.read_csv(f'{out_path}/cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv')
    
    clustered_df = pd.read_csv(f'{out_path}/cell_by_gene_matrix_dilate10_20240718_withbarcodes_clustering_10bcs_0.4thresh.csv')

    print(roi_name)
    df = unclustered_df.merge(clustered_df[['cell_id','bc_cluster','bc_cluster_n_bcs','bc_cluster_found_bcs','bc_cluster_bc_names']],on=['cell_id'],how='left')   
    
    roi_barcodes[roi_name] = df
    roi_barcodes[roi_name]['roi'] = roi_name



roi_1
roi_2
roi_3
timezero_roi_1
timezero_roi_2
timezero_roi_3
run2_roi_2
run2_roi_3


In [5]:
roi_barcodes

{'roi_1':        object_id  cell_id  n_called_barcodes  barcode_names  \
 0              3        3                  0            NaN   
 1              2        2                  2  bc_043-bc_094   
 2              4        4                  0            NaN   
 3              5        5                  0            NaN   
 4              6        6                  0            NaN   
 ...          ...      ...                ...            ...   
 39626      39630    39630                  1         bc_092   
 39627      39631    39631                  0            NaN   
 39628      39627    39627                  2  bc_044-bc_092   
 39629      39629    39629                  2  bc_044-bc_092   
 39630      39628    39628                  2  bc_044-bc_092   
 
             called_barcodes          area      center_x      center_y  \
 0                        []   6185.884792  35953.336715   4743.492535   
 1      ['bc_043', 'bc_094']   8039.385231  36134.087539   4746.507098   

In [61]:
# let's only bother calculating this stuff for clusters with at least 2 barcodes:
roi_barcodes_many = {}

many_bc_cutoff = 2
number_of_cells_cutoff = 0

for roi_name,roi_df in roi_barcodes.items():
    roi_barcodes_many[roi_name] = roi_df[roi_df['bc_cluster_n_bcs'] > many_bc_cutoff]
    roi_barcodes_many[roi_name] = roi_barcodes_many[roi_name][roi_barcodes_many[roi_name].groupby('bc_cluster_bc_names').barcode_names.transform(len) > number_of_cells_cutoff]


In [62]:
len(np.unique(roi_barcodes_many['timezero_roi_3']['bc_cluster'].values))

2085

In [65]:
len(np.unique(roi_barcodes_many['timezero_roi_2']['bc_cluster'].values))

4146

In [67]:
len(np.unique(roi_barcodes_many['roi_2']['bc_cluster'].values))

1398

In [64]:
roi_barcodes_many['roi_2']

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
31,31,6.0,0.0,9.0,0.0,6.0,0.0,1.0,9.0,9.0,...,POLYGON ((15143.759652654107 1657.077221232863...,POINT (15147.005205115243 1714.8636184912655),15147.005205,1714.863618,12799.116430,2386.0,4.0,"['bc_025', 'bc_060', 'bc_062', 'bc_066']",bc_025-bc_060-bc_062-bc_066,roi_2
35,34,0.0,0.0,9.0,0.0,1.0,0.0,0.0,5.0,3.0,...,POLYGON ((15227.36882594208 1664.3523617876226...,POINT (15248.757498497193 1719.2191613291047),15248.757498,1719.219161,11641.532047,2386.0,4.0,"['bc_025', 'bc_060', 'bc_062', 'bc_066']",bc_025-bc_060-bc_062-bc_066,roi_2
36,35,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,...,POLYGON ((15525.307878083491 1717.613536810560...,POINT (15592.614506473617 1723.6034386746485),15592.614506,1723.603439,10664.189218,2386.0,4.0,"['bc_025', 'bc_060', 'bc_062', 'bc_066']",bc_025-bc_060-bc_062-bc_066,roi_2
40,50,4.0,0.0,8.0,0.0,3.0,0.0,0.0,6.0,6.0,...,POLYGON ((16986.356010126947 1707.136060761678...,POINT (16985.062097938822 1750.1873440987265),16985.062098,1750.187344,9865.242028,2386.0,4.0,"['bc_025', 'bc_060', 'bc_062', 'bc_066']",bc_025-bc_060-bc_062-bc_066,roi_2
43,38,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,14.0,...,POLYGON ((16719.582569369133 1688.602065765115...,POINT (16734.212951550206 1742.1249960494295),16734.212952,1742.124996,8222.084161,2386.0,4.0,"['bc_025', 'bc_060', 'bc_062', 'bc_066']",bc_025-bc_060-bc_062-bc_066,roi_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67095,67065,2.0,0.0,9.0,0.0,10.0,0.0,0.0,12.0,37.0,...,"POLYGON ((26707.47942815746 45765.1162830235, ...",POINT (26725.052161809483 45869.62785558939),26725.052162,45869.627856,35709.256507,274.0,4.0,"['bc_041', 'bc_045', 'bc_047', 'bc_052']",bc_041-bc_045-bc_047-bc_052,roi_2
67098,67107,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,...,POLYGON ((27645.126521144335 45904.42173714778...,POINT (27660.106245153052 45941.16244696866),27660.106245,45941.162447,6480.112546,274.0,4.0,"['bc_041', 'bc_045', 'bc_047', 'bc_052']",bc_041-bc_045-bc_047-bc_052,roi_2
67105,67106,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,15.0,...,POLYGON ((28043.57464374964 45897.298574998546...,POINT (28080.90330775961 45951.41776027996),28080.903308,45951.417760,15172.993162,274.0,4.0,"['bc_041', 'bc_045', 'bc_047', 'bc_052']",bc_041-bc_045-bc_047-bc_052,roi_2
67107,67096,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,6.0,...,POLYGON ((26980.513167019493 45858.83772233983...,POINT (26998.581369712 45938.179157070015),26998.581370,45938.179157,12428.107684,274.0,4.0,"['bc_041', 'bc_045', 'bc_047', 'bc_052']",bc_041-bc_045-bc_047-bc_052,roi_2


In [251]:
# normalize to sum of barcode counts
def calculate_normalized_matrix(matrix):
    return matrix.div(matrix.sum(axis=1),axis=0)

def find_possible_locations(roi,D_small_locs):
    
    possible_cluster_locs = {}
    for cluster,ix in roi.groupby('bc_cluster').groups.items():
        locs = np.where(roi.index.isin(ix))[0]
        
        # if any of points are within distance, we should include them in the next step
        if np.any(np.isin(D_small_locs,locs)):
            possible_cluster_locs[cluster] = locs

    return possible_cluster_locs



In [252]:
big_clusters = roi_barcodes['roi_1']['bc_cluster'].values

dist_thresh = 0.4

clusters_to_merge = []

for roi_a,roi_b in combinations(roi_barcodes_many.keys(),2):
    print(roi_a,roi_b)

    matrix_a = calculate_normalized_matrix(roi_barcodes_many[roi_a][barcode_cols])
    matrix_b = calculate_normalized_matrix(roi_barcodes_many[roi_b][barcode_cols])

    # calculate the distance matrix first. this is actually not the most inefficient part of the code and we want to avoid repeated computations
    D  = scipy.spatial.distance.cdist(roi_barcodes_many[roi_a][barcode_cols].values,
                            roi_barcodes_many[roi_b][barcode_cols].values,metric='braycurtis')

    # pre-process to only include clusters where any of the points are within threshold distance
    # this cuts down on number of pairwise comparisons we need to make

    # find the locations of the pairwise distances that are less than the threshold
    D_small_locs = np.where(D < dist_thresh)
    print('Distance has been calculated')

    # get the subset clusters and locations that could possibly need to be merged.
    possible_cluster_a_locs = find_possible_locations(roi_barcodes_many[roi_a],D_small_locs[0])
    possible_cluster_b_locs = find_possible_locations(roi_barcodes_many[roi_b],D_small_locs[1])

    # now we need to calculate the pairwise distances between the possible clusters
    for cluster_a,locs_a in possible_cluster_a_locs.items():
        for cluster_b,locs_b in possible_cluster_b_locs.items():
            # print(cluster_a,cluster_b)

            this_d = np.mean(D[locs_a,:][:,locs_b])
            if this_d < dist_thresh:
                # print(cluster_a,cluster_b)
                clusters_to_merge.append((roi_a,roi_b,cluster_a,cluster_b))

roi_1 roi_2
Distance has been calculated
roi_1 roi_3
Distance has been calculated
roi_1 timezero_roi_1
Distance has been calculated
roi_1 timezero_roi_2
Distance has been calculated
roi_1 timezero_roi_3
Distance has been calculated
roi_1 run2_roi_2
Distance has been calculated
roi_1 run2_roi_3
Distance has been calculated
roi_2 roi_3
Distance has been calculated
roi_2 timezero_roi_1
Distance has been calculated
roi_2 timezero_roi_2
Distance has been calculated
roi_2 timezero_roi_3
Distance has been calculated
roi_2 run2_roi_2
Distance has been calculated
roi_2 run2_roi_3
Distance has been calculated
roi_3 timezero_roi_1
Distance has been calculated
roi_3 timezero_roi_2
Distance has been calculated
roi_3 timezero_roi_3
Distance has been calculated
roi_3 run2_roi_2
Distance has been calculated
roi_3 run2_roi_3
Distance has been calculated
timezero_roi_1 timezero_roi_2
Distance has been calculated
timezero_roi_1 timezero_roi_3
Distance has been calculated
timezero_roi_1 run2_roi_2
Distanc

In [253]:
clusters_to_merge

[('roi_1', 'roi_2', 1088.0, 3272.0),
 ('roi_1', 'roi_2', 1362.0, 1527.0),
 ('roi_1', 'roi_2', 1993.0, 2243.0),
 ('roi_1', 'roi_3', 1164.0, 506.0),
 ('roi_1', 'roi_3', 1858.0, 418.0),
 ('roi_1', 'timezero_roi_1', 1238.0, 2676.0),
 ('roi_1', 'timezero_roi_1', 1895.0, 3299.0),
 ('roi_1', 'timezero_roi_1', 2367.0, 4105.0),
 ('roi_1', 'timezero_roi_2', 948.0, 1288.0),
 ('roi_1', 'timezero_roi_2', 1285.0, 2235.0),
 ('roi_1', 'timezero_roi_2', 1771.0, 879.0),
 ('roi_1', 'timezero_roi_2', 1914.0, 1697.0),
 ('roi_1', 'timezero_roi_2', 2144.0, 4368.0),
 ('roi_1', 'timezero_roi_2', 2336.0, 2666.0),
 ('roi_1', 'timezero_roi_3', 516.0, 2182.0),
 ('roi_1', 'timezero_roi_3', 1135.0, 2316.0),
 ('roi_1', 'timezero_roi_3', 1840.0, 2455.0),
 ('roi_1', 'timezero_roi_3', 1978.0, 1550.0),
 ('roi_1', 'timezero_roi_3', 2025.0, 1687.0),
 ('roi_1', 'timezero_roi_3', 2131.0, 2332.0),
 ('roi_1', 'run2_roi_2', 979.0, 2999.0),
 ('roi_1', 'run2_roi_2', 1001.0, 2805.0),
 ('roi_1', 'run2_roi_2', 1061.0, 3185.0),
 ('ro

In [259]:
for (roi_a,roi_b,cluster_a,cluster_b) in clusters_to_merge:
    print(roi_a,roi_b,cluster_a,cluster_b)

    a_cells = roi_barcodes_many[roi_a][roi_barcodes_many[roi_a]['bc_cluster'] == cluster_a]
    b_cells = roi_barcodes_many[roi_b][roi_barcodes_many[roi_b]['bc_cluster'] == cluster_b]
    print(a_cells['bc_cluster_bc_names'].values[0])
    print(b_cells['bc_cluster_bc_names'].values[0])

    print(np.mean(scipy.spatial.distance.pdist(calculate_normalized_matrix(a_cells[barcode_cols]),metric='braycurtis')))
    print(np.mean(scipy.spatial.distance.pdist(calculate_normalized_matrix(b_cells[barcode_cols]),metric='braycurtis')))

    print(np.mean(scipy.spatial.distance.cdist(calculate_normalized_matrix(a_cells[barcode_cols]),
                                               calculate_normalized_matrix(b_cells[barcode_cols]),metric='braycurtis')))

    joint_initial_barocdes = np.unique(ast.literal_eval(a_cells['bc_cluster_found_bcs'].values[0]) + ast.literal_eval(b_cells['bc_cluster_found_bcs'].values[0]))
    

    print(a_cells[joint_initial_barocdes])
    print(b_cells[joint_initial_barocdes])
    

    # joint_initial_barcodes = roi_barcodes_many[roi_a][roi_barcodes_many[roi_a]['bc_cluster'] == cluster_a]['bc_cluster_found_bcs']+  roi_barcodes_many[roi_b][roi_barcodes_many[roi_b]['bc_cluster'] == cluster_b]['bc_cluster_found_bcs']+ 
    # print(joint_initial_barcodes)

roi_1 roi_2 1088.0 3272.0
bc_022-bc_044-bc_095
bc_022-bc_041-bc_095
0.09523809523809522
nan
0.40142857142857136
       bc_022  bc_041  bc_044  bc_095
11261     6.0     0.0     3.0     5.0
11324     6.0     0.0     4.0     8.0
       bc_022  bc_041  bc_044  bc_095
11694     5.0     4.0     1.0     9.0
roi_1 roi_2 1362.0 1527.0
bc_008-bc_033-bc_045-bc_049-bc_073-bc_092
bc_045-bc_073-bc_076-bc_092
nan
nan
0.35593220338983045
       bc_008  bc_033  bc_045  bc_049  bc_073  bc_076  bc_092
35733     5.0     5.0     8.0     6.0     5.0     1.0    24.0
       bc_008  bc_033  bc_045  bc_049  bc_073  bc_076  bc_092
37099     0.0     0.0     8.0     0.0     5.0     9.0    25.0
roi_1 roi_2 1993.0 2243.0
bc_015-bc_033-bc_060
bc_015-bc_033-bc_076
nan
0.2122762148337596
0.32974059188892946
       bc_015  bc_033  bc_060  bc_076
12040     7.0    10.0     5.0     0.0
       bc_015  bc_033  bc_060  bc_076
57426     6.0     6.0     0.0     4.0
58916     6.0     9.0     2.0     4.0
roi_1 roi_3 1164.0 506.0


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _method

bc_035-bc_059-bc_066-bc_095
bc_004-bc_059-bc_066-bc_083-bc_095
nan
nan
0.37400530503978785
       bc_004  bc_035  bc_059  bc_066  bc_083  bc_095
52844     2.0     4.0     7.0     4.0     0.0     4.0
       bc_004  bc_035  bc_059  bc_066  bc_083  bc_095
30419       4       1       6       7       4       6
roi_2 run2_roi_3 1197.0 7948.0
bc_024-bc_035-bc_052-bc_057-bc_073
bc_024-bc_025-bc_035-bc_052-bc_063-bc_073
nan
nan
0.34157509157509164
       bc_024  bc_025  bc_035  bc_052  bc_057  bc_063  bc_073
53079     4.0     3.0    19.0     7.0     4.0     0.0     8.0
      bc_024  bc_025  bc_035  bc_052  bc_057  bc_063  bc_073
3880       5      10      17       6       1       6       6
roi_2 run2_roi_3 2128.0 9554.0
bc_001-bc_041-bc_081
bc_001-bc_041-bc_066-bc_081
nan
nan
0.43396226415094347
      bc_001  bc_041  bc_066  bc_081
9089     6.0    11.0     0.0    13.0
       bc_001  bc_041  bc_066  bc_081
69708       6      11       6      13
roi_2 run2_roi_3 2347.0 9300.0
bc_001-bc_022-bc_024
b

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _method

0.10342263883348846
0.21293558863070114
      bc_060  bc_070  bc_076  bc_079
996      0.0    16.0    34.0    25.0
5817     0.0     0.0    21.0    26.0
5881     3.0     0.0    22.0    20.0
      bc_060  bc_070  bc_076  bc_079
4134     9.0     0.0    32.0    27.0
4248     0.0     0.0    19.0    19.0
4251     8.0     0.0    54.0    47.0
timezero_roi_1 timezero_roi_2 891.0 2102.0
bc_025-bc_047-bc_060-bc_073-bc_092
bc_017-bc_047-bc_060-bc_076-bc_092
0.28921568627450983
0.24345445921673314
0.28873660655181405
      bc_017  bc_025  bc_047  bc_060  bc_073  bc_076  bc_092
6738     0.0     1.0    48.0    39.0     4.0     0.0    37.0
6875     0.0    12.0    14.0    22.0    13.0     0.0    20.0
      bc_017  bc_025  bc_047  bc_060  bc_073  bc_076  bc_092
338      0.0     0.0    22.0    34.0     0.0     6.0    24.0
386      0.0     0.0    51.0    57.0     0.0     8.0    43.0
1461    15.0     0.0    31.0    18.0     0.0     0.0    29.0
1563     5.0     0.0    18.0    20.0     0.0     0.0    38.0
tim

...

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _method

nan
0.28
      bc_029  bc_059  bc_060  bc_080
1533    16.0    12.0     4.0     0.0
       bc_029  bc_059  bc_060  bc_080
29623       9       8       1       5
timezero_roi_2 run2_roi_2 3526.0 2411.0
bc_041-bc_060-bc_063-bc_074
bc_041-bc_074-bc_091
0.2780612244897959
nan
0.3589981447124304
       bc_041  bc_060  bc_063  bc_074  bc_091
4851     12.0     9.0     0.0    27.0     0.0
12307    10.0     0.0    11.0    24.0     0.0
       bc_041  bc_060  bc_063  bc_074  bc_091
46664      10       0       0      19       9
timezero_roi_2 run2_roi_2 3629.0 2004.0
bc_039-bc_055-bc_091
bc_010-bc_039-bc_055-bc_085
0.13472706155632985
nan
0.2698412698412699
       bc_010  bc_039  bc_055  bc_085  bc_091
18790     0.0    46.0    28.0     0.0     7.0
18869     0.0    44.0    40.0     0.0     0.0
       bc_010  bc_039  bc_055  bc_085  bc_091
33489       9      51      41      23       0
timezero_roi_2 run2_roi_2 3677.0 1916.0
bc_008-bc_040-bc_087
bc_008-bc_016-bc_021-bc_040-bc_049-bc_055-bc_079
0.153630

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _method

bc_026-bc_045-bc_060-bc_073
bc_041-bc_060-bc_073
nan
nan
0.3939393939393939
     bc_026  bc_041  bc_045  bc_060  bc_073
758     4.0     0.0     4.0    12.0     6.0
       bc_026  bc_041  bc_045  bc_060  bc_073
49586       0       5       0      13       5
timezero_roi_2 run2_roi_3 4806.0 205.0
bc_027-bc_073-bc_078-bc_095
bc_057-bc_073-bc_095
nan
0.3147607873055055
0.3279787965594709
      bc_027  bc_057  bc_073  bc_078  bc_095
1972     6.0     0.0    26.0    10.0    37.0
       bc_027  bc_057  bc_073  bc_078  bc_095
4898        0       9      34       0      45
19770       0       0       9       0      27
20385       0       0      12       0      15
20626       0       0      24       0      43
41147       0      11       9       0      27
49205       0       2      11       0      38
50906       0       0      17       0      37
timezero_roi_3 run2_roi_2 611.0 2551.0
bc_060-bc_070-bc_086
bc_002-bc_060-bc_070
0.23420289855072465
nan
0.3447513498152885
      bc_002  bc_060  bc_070  bc

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _method

In [189]:
roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1088.0][['bc_022','bc_041','bc_095','bc_044']]

,bc_022,bc_041,bc_095,bc_044
11261,6.0,0.0,5.0,3.0
11324,6.0,0.0,8.0,4.0


In [190]:
roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==3272.0][['bc_022','bc_041','bc_095','bc_044']]

,bc_022,bc_041,bc_095,bc_044
11694,5.0,4.0,9.0,1.0


In [194]:
roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1362.0][['bc_008', 'bc_033', 'bc_045', 'bc_049', 'bc_073', 'bc_092']]

,bc_008,bc_033,bc_045,bc_049,bc_073,bc_092
35733,5.0,5.0,8.0,6.0,5.0,24.0


In [193]:
[['bc_008', 'bc_033', 'bc_045', 'bc_049', 'bc_073', 'bc_092']]roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==1527.0]

,bc_008,bc_033,bc_045,bc_049,bc_073,bc_092
37099,0.0,0.0,8.0,0.0,5.0,25.0


In [198]:
a= roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1362.0][barcode_cols]
a = a.div(a.sum(axis=1),axis=0)

b = roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==1527.0][barcode_cols]
b = b.div(b.sum(axis=1),axis=0)
scipy.spatial.distance.cdist(a,b,metric='braycurtis')

array([[0.3559322]])

In [199]:
a= roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1088.0][barcode_cols]
a = a.div(a.sum(axis=1),axis=0)

b = roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==3272.0][barcode_cols]
b = b.div(b.sum(axis=1),axis=0)
scipy.spatial.distance.cdist(a,b,metric='braycurtis')

array([[0.40285714],
       [0.4       ]])

In [204]:
a= roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1993.0][barcode_cols]
a = a.div(a.sum(axis=1),axis=0)

b = roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==2243.0][barcode_cols]
b = b.div(b.sum(axis=1),axis=0)
scipy.spatial.distance.cdist(a,b,metric='braycurtis')

array([[0.39705882, 0.26242236]])

In [207]:
roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==1993.0][['bc_015', 'bc_033', 'bc_060','bc_076']]

,bc_015,bc_033,bc_060,bc_076
12040,7.0,10.0,5.0,0.0


In [208]:
roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==2243.0][['bc_015', 'bc_033', 'bc_060','bc_076']]

,bc_015,bc_033,bc_060,bc_076
57426,6.0,6.0,0.0,4.0
58916,6.0,9.0,2.0,4.0


In [56]:
np.mean(scipy.spatial.distance.cdist(cluster_a_cells[barcode_cols].values,
                                              cluster_b_cells[barcode_cols].values,
                                              metric='braycurtis'))

0.942738272453147

In [148]:


for roi in ['roi_1','roi_2','roi_3','run2_roi_2','run2_roi_3']:


In [227]:
roi = 'roi_2'
n_cells = 100



for bc_cluster,size in roi_barcodes_many[roi].groupby('bc_cluster_names').size().sort_values(ascending=False):
    print(bc_cluster,size)
    

KeyError: 'bc_cluster_names'

In [231]:
roi_barcodes_many[roi].groupby(['bc_cluster','bc_cluster_bc_names']).size().sort_values(ascending=False)

bc_cluster  bc_cluster_bc_names                      
72.0        bc_001-bc_029-bc_031-bc_087                  1392
372.0       bc_029-bc_059-bc_064-bc_068-bc_085           1152
173.0       bc_041-bc_066-bc_093                          431
2386.0      bc_025-bc_060-bc_062-bc_066                   390
42.0        bc_030-bc_040-bc_058-bc_059-bc_083-bc_095     377
                                                         ... 
2125.0      bc_004-bc_041-bc_063-bc_085                     1
2128.0      bc_001-bc_041-bc_081                            1
2138.0      bc_028-bc_067-bc_087-bc_095                     1
2146.0      bc_048-bc_059-bc_068-bc_081-bc_086              1
3423.0      bc_024-bc_029-bc_064-bc_092                     1
Length: 1398, dtype: int64

In [232]:
roi_barcodes_many[roi][roi_barcodes_many[roi]['bc_cluster'] == 72.0 ]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
26655,26664,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,...,POLYGON ((9520.753049524455 20229.191311905568...,POINT (9538.988936988937 20280.984168810257),9538.988937,20280.984169,11732.905640,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
26878,26900,1.0,0.0,0.0,0.0,3.0,0.0,0.0,7.0,8.0,...,POLYGON ((8134.592424086865 20427.588215246233...,POINT (8157.66480161237 20482.75883644677),8157.664802,20482.758836,8722.151685,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
27554,27575,1.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,9.0,...,POLYGON ((13316.641600271756 21004.50609938148...,POINT (13342.538047138047 21039.330527497194),13342.538047,21039.330527,7231.212595,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
28559,28590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,"POLYGON ((15188.01725006864 21711.45520021965,...",POINT (15166.387693469233 21751.55141562854),15166.387693,21751.551416,7478.286782,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
28641,28626,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,...,POLYGON ((10478.773611033657 21755.08425334261...,POINT (10515.107159757712 21795.793961764146),10515.107160,21795.793962,10471.268295,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50820,50756,3.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,29.0,...,POLYGON ((6913.0496280979005 34942.00496280979...,POINT (6942.328405868913 35008.42791862802),6942.328406,35008.427919,9567.497799,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
50843,50841,2.0,0.0,14.0,0.0,0.0,0.0,0.0,1.0,2.0,...,POLYGON ((10288.265376556366 34980.80768079481...,POINT (10274.622249164187 35034.52037179911),10274.622249,35034.520372,7401.183043,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
50852,50769,4.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,9.0,...,POLYGON ((11264.414953921314 34939.47423305293...,POINT (11274.178559929163 35020.65618208162),11274.178560,35020.656182,12128.936597,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2
51156,51138,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,...,POLYGON ((10087.060807014208 35157.80854969982...,POINT (10088.51535603504 35222.44779629819),10088.515356,35222.447796,7939.735471,72.0,4.0,"['bc_001', 'bc_029', 'bc_031', 'bc_087']",bc_001-bc_029-bc_031-bc_087,roi_2


In [249]:
# focal_cluster = 72.0
focal_roi = 'roi_2'
n_clusters = 100

cluster_names = roi_barcodes_many[focal_roi].groupby('bc_cluster_bc_names').size().sort_values(ascending=False).index[:n_clusters]
# cluster_names = roi_barcodes_many[focal_roi].groupby('barcode_names').size().sort_values(ascending=False).index[:n_clusters]

for roi in ['timezero_roi_1','timezero_roi_2','timezero_roi_3']:

    subset = roi_barcodes_many[roi][roi_barcodes_many[roi]['barcode_names'].isin(cluster_names)]
    print(subset['barcode_names'].values)
    # print(roi)s
#     for bc_cluster,size in roi_barcodes_many[roi].groupby('bc_cluster_bc_names').size().sort_values(ascending=False).items():
#         print(bc_cluster,size)

# roi_barcodes_many[roi]['bc_cluster_bc_names']

[]
['bc_004-bc_047-bc_086' 'bc_025-bc_033-bc_060']
[]


In [250]:
roi_barcodes_many['timezero_roi_2'][roi_barcodes_many['timezero_roi_2']['barcode_names'] == 'bc_025-bc_033-bc_060']

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
11751,11751,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,23.0,...,"POLYGON ((12518.96796342723 33489.47074219987,...",POINT (12529.215670592797 33577.62636323978),12529.215671,33577.626363,18396.765357,1177.0,4.0,"['bc_011', 'bc_025', 'bc_033', 'bc_075']",bc_011-bc_025-bc_033-bc_075,timezero_roi_2


In [237]:
cluster_names = roi_barcodes_many['roi_2'].groupby('bc_cluster_bc_names').size().sort_values(ascending=False).index[:20]


In [244]:
list(roi_barcodes_many[roi].columns)

['object_id',
 'AKT1',
 'AXL',
 'BGN',
 'BIRC3',
 'BIRC5',
 'BMP2',
 'BMP4',
 'CALD1',
 'CAPG',
 'CCNA2',
 'CCND1',
 'CD36',
 'CDK1',
 'CDKN1A',
 'CDKN1B',
 'COL1A1',
 'COL5A2',
 'COL9A3',
 'CTNNB1',
 'DCN',
 'DKK3',
 'EGFR',
 'ENSMUSG00000071361',
 'ERBB3',
 'ESPN',
 'FGF1',
 'FGFR1',
 'FN1',
 'FOS',
 'FOSB',
 'FOSL1',
 'FOSL2',
 'FOXF2',
 'GFP',
 'GPX8',
 'GTSE1',
 'HIF1A',
 'HK2',
 'IFIT2',
 'IGFBP2',
 'IGFBP7',
 'IRX3',
 'ITGA3',
 'ITGA5',
 'ITGA6',
 'ITGA8',
 'ITGAV',
 'JUN',
 'KIT',
 'KLF4',
 'L1CAM',
 'LMOD1',
 'LOXL2',
 'MAPK14',
 'MEF2C',
 'MERTK',
 'MET',
 'MITF',
 'MKI67',
 'MLANA',
 'MMP1',
 'MMP11',
 'MT-RNR2',
 'MYBL2',
 'MYC',
 'MYLK',
 'MYOCD',
 'NANOG',
 'NCAM1',
 'NFATC2',
 'NFE2L2',
 'NGFR',
 'NOTCH3',
 'NOTCH4',
 'NRG1',
 'NT5E',
 'OASL',
 'PDGFA',
 'PDGFC',
 'PDGFRB',
 'PKDCC',
 'PRKCA',
 'PRRX1',
 'RGS2',
 'RIGI',
 'ROR1',
 'ROR2',
 'RUNX2',
 'SCG2',
 'SERPINE1',
 'SERPINF1',
 'SFRP1',
 'SFRP4',
 'SLC7A8',
 'SLIT2',
 'SMAD3',
 'SNAI1',
 'SNAI2',
 'SOX10',
 'SOX2',

In [162]:
len(D_small_locs[1])

15650

In [156]:
D[np.where(D < 0.5)].shape

(85210,)

In [264]:
roi_barcodes_many['roi_3'][roi_barcodes_many['roi_3']['barcode_names'].isin(['bc_003-bc_024-bc_059-bc_077-bc_085',
                                     'bc_024-bc_025-bc_059-bc_077-bc_085',
                                     'bc_024-bc_025-bc_059-bc_085',
                                    'bc_025-bc_059-bc_077-bc_085' ])]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
10290,10299,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,...,"POLYGON ((18746.55864254217 11840.35088801883,...",POINT (18757.322531265345 11897.962539352453),18757.322531,11897.962539,9036.813767,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
10645,10641,1.0,0.0,0.0,0.0,7.0,0.0,0.0,8.0,20.0,...,POLYGON ((21621.928932188133 12022.92893218813...,POINT (21597.35632117015 12073.060835321734),21597.356321,12073.060835,12934.105481,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
11117,11093,4.0,0.0,0.0,1.0,4.0,0.0,1.0,15.0,30.0,...,POLYGON ((20755.830695421813 12293.23812939816...,POINT (20777.758974358974 12353.521323312701),20777.758974,12353.521323,15504.141736,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
11321,11233,6.0,1.0,0.0,0.0,3.0,0.0,0.0,11.0,32.0,...,POLYGON ((21366.40026526552 12373.343842414793...,POINT (21363.30923458725 12453.721486415881),21363.309235,12453.721486,17240.356214,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
11553,11546,1.0,0.0,0.0,0.0,4.0,0.0,0.0,12.0,4.0,...,"POLYGON ((21366 12547, 21352.579708741796 1256...",POINT (21355.20989383892 12612.313937922232),21355.209894,12612.313938,8541.481706,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20765,20784,4.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,3.0,...,POLYGON ((23718.753049524457 17391.19131190556...,POINT (23701.956361966008 17443.852625937834),23701.956362,17443.852626,9895.711518,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
20882,20834,7.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,10.0,...,POLYGON ((24198.425070742873 17415.85504244572...,POINT (24247.827698087433 17478.69537071535),24247.827698,17478.695371,15047.507432,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
20903,20845,2.0,0.0,1.0,0.0,2.0,0.0,0.0,9.0,7.0,...,POLYGON ((23884.22879492601 17495.200394859046...,POINT (23936.569949186338 17487.895199502585),23936.569949,17487.895200,11545.066278,59.0,3.0,"['bc_024', 'bc_059', 'bc_085']",bc_024-bc_059-bc_085,roi_3
21141,21194,2.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,6.0,...,POLYGON ((23935.68379472694 17588.271937853147...,POINT (23937.969176387414 17635.960527866147),23937.969176,17635.960528,10589.159985,372.0,3.0,"['bc_025', 'bc_059', 'bc_085']",bc_025-bc_059-bc_085,roi_3


In [177]:
len(roi_barcodes_many[roi_a].groupby('bc_cluster').groups.keys())

935

In [262]:
roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster'] == 16]['SFRP1'].values

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 2., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 2., 1., 0., 0., 1., 0.,
       0., 1., 0., 2., 0., 2., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0.])

In [178]:
len(roi_barcodes_many[roi_b].groupby('bc_cluster').groups.keys())

1398

In [179]:
len(possible_cluster_b_locs.keys())

402

In [180]:

    
    # print(locs)
    

In [110]:
ix_a

Int64Index([28794, 28927, 29423, 29678, 29960], dtype='int64')

In [111]:
ix_b

Int64Index([33886], dtype='int64')

In [112]:
locs_a

array([6358, 6396, 6532, 6587, 6664])

In [113]:
locs_b

array([6065])

In [99]:
np.where(roi_barcodes_many[roi_b].isin(ix_b))

(array([10784, 10784, 11922, 11922]), array([  0, 216,   0, 216]))

In [114]:
list(itertools.product(locs_a,locs_b))

[(6358, 6065), (6396, 6065), (6532, 6065), (6587, 6065), (6664, 6065)]

In [128]:
np.ix_(itertools.product(locs_a,locs_b))

ValueError: Cross index must be 1 dimensional

In [131]:

D[zip(locs_a,locs_b)]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [133]:
tuple(itertools.product(locs_a,locs_b))

[(6358, 6065), (6396, 6065), (6532, 6065), (6587, 6065), (6664, 6065)]

(5, 1)

In [138]:
D[(0,1),(1,0)]

array([32.52691193, 29.44486373])

In [126]:
D[np.meshgrid(locs_a,locs_b)].shape

(2, 1, 5, 15541)

In [123]:
D[list(itertools.product(locs_a,locs_b))[:,None],list(itertools.product(locs_a,locs_b))[None,:]]

TypeError: list indices must be integers or slices, not tuple

In [120]:
D.shape

(8499, 15541)

In [122]:
D[0:2,1:5]

array([[32.52691193, 28.0713377 , 36.3868108 , 21.65640783],
       [23.23790008, 18.49324201, 26.41968963, 24.4744765 ]])

In [95]:
roi_barcodes_many[roi_b].shape

(15541, 231)

In [94]:
roi_barcodes_many[roi_a].shape

(8499, 227)

In [92]:
list(itertools.product(locs_a,locs_b))

[(6359, 6359),
 (6359, 6359),
 (6359, 6400),
 (6359, 6400),
 (6359, 6536),
 (6359, 6536),
 (6359, 6594),
 (6359, 6594),
 (6359, 6359),
 (6359, 6359),
 (6359, 6400),
 (6359, 6400),
 (6359, 6536),
 (6359, 6536),
 (6359, 6594),
 (6359, 6594),
 (6400, 6359),
 (6400, 6359),
 (6400, 6400),
 (6400, 6400),
 (6400, 6536),
 (6400, 6536),
 (6400, 6594),
 (6400, 6594),
 (6400, 6359),
 (6400, 6359),
 (6400, 6400),
 (6400, 6400),
 (6400, 6536),
 (6400, 6536),
 (6400, 6594),
 (6400, 6594),
 (6536, 6359),
 (6536, 6359),
 (6536, 6400),
 (6536, 6400),
 (6536, 6536),
 (6536, 6536),
 (6536, 6594),
 (6536, 6594),
 (6536, 6359),
 (6536, 6359),
 (6536, 6400),
 (6536, 6400),
 (6536, 6536),
 (6536, 6536),
 (6536, 6594),
 (6536, 6594),
 (6594, 6359),
 (6594, 6359),
 (6594, 6400),
 (6594, 6400),
 (6594, 6536),
 (6594, 6536),
 (6594, 6594),
 (6594, 6594),
 (6594, 6359),
 (6594, 6359),
 (6594, 6400),
 (6594, 6400),
 (6594, 6536),
 (6594, 6536),
 (6594, 6594),
 (6594, 6594)]

In [79]:
roi_barcodes_many[roi_b].groupby('bc_cluster').groups

{7.0: [40865, 43536, 43691, 44916, 45503, 45566, 46931, 47989, 50541], 8.0: [195, 217, 224, 241, 246, 247, 249, 300, 318, 361, 391, 399, 402, 440, 474, 482, 501, 522, 533, 636, 687, 689, 690, 737, 741, 783, 809, 857, 956, 966, 1072, 1092, 1104, 1133, 1269, 1295, 1329, 1374, 2187], 9.0: [32134, 34174, 35365, 35368], 10.0: [20102, 40414, 40636, 40675, 40892, 40955, 41190, 41285, 41351, 42135, 44003, 44131, 44875, 45204, 45416, 45419, 46388, 46406, 46411, 46575, 47438, 47550, 47607, 47825, 48034, 48212, 48233, 48381, 48430, 48661, 48684, 48861, 48919, 49279, 49418, 49568, 49661, 49671, 50388, 50979, 51097, 51143, 51310, 51655, 51968, 53731, 54415, 54655, 54723], 11.0: [11611, 49248], 13.0: [48916, 49681], 15.0: [7099, 32470, 32680, 32745, 33029, 33079, 33115, 33206, 33245, 33637, 33723, 33858, 33861, 33875, 33877, 33890, 33907, 33911, 33950, 33992, 34004, 34026, 34031, 34070, 34075, 34078, 34105, 34115, 34129, 34130, 34179, 34187, 34188, 34196, 34202, 34264, 34269, 34274, 34283, 34286, 34

In [58]:
roi_barcodes_many['roi_1'][roi_barcodes_many['roi_1']['bc_cluster']==81.0]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
35066,35064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,4.0,...,POLYGON ((10378.286093236458 50234.71523309115...,POINT (10415.93543735457 50287.270868154556),10415.935437,50287.270868,8278.401904,81.0,2.0,"['bc_014', 'bc_024']",bc_014-bc_024,roi_1
35524,35504,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,...,POLYGON ((10222.181764336518 50919.76701495278...,POINT (10278.44154294171 50942.327938766095),10278.441543,50942.327939,14711.000184,81.0,2.0,"['bc_014', 'bc_024']",bc_014-bc_024,roi_1


In [59]:
roi_barcodes_many['roi_2'][roi_barcodes_many['roi_2']['bc_cluster']==839.0]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,nucleus_dilated,nucleus_centroid,center_x,center_y,area,bc_cluster,bc_cluster_n_bcs,bc_cluster_found_bcs,bc_cluster_bc_names,roi
17635,17649,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,...,POLYGON ((32831.36882594208 13859.352361787624...,POINT (32841.147061709016 13912.946466529318),32841.147062,13912.946467,8013.820695,839.0,2.0,"['bc_014', 'bc_024']",bc_014-bc_024,roi_2
21621,21675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"POLYGON ((33464.5165954707 16128.110636471318,...",POINT (33465.37831314727 16164.518474635772),33465.378313,16164.518475,6461.727686,839.0,2.0,"['bc_014', 'bc_024']",bc_014-bc_024,roi_2
41336,41252,2.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0,...,POLYGON ((6506.4075424979965 29483.20280952368...,POINT (6491.202659704659 29558.00515351772),6491.202660,29558.005154,17200.081943,839.0,2.0,"['bc_014', 'bc_024']",bc_014-bc_024,roi_2
